In [ ]:
import configparser
import datetime as dt
import json
import random
import schedule
import time

from kafka import KafkaProducer
from schedule import every, repeat, run_pending

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')

In [ ]:
TOPIC = config['LOCAL']['TOPIC']
MACHINE_PREFIX = config['LOCAL']['MACHINE_PREFIX']
SERVER = config['LOCAL']['SERVER']
INTERVAL = int(config['LOCAL']['INTERVAL'])

In [ ]:
MACHINES = [MACHINE_PREFIX + str(i) for i in range(10, 60, 10)]

In [ ]:
def serializer(data):
    return json.dumps(data).encode('utf-8')

In [ ]:
producer = KafkaProducer(
    # bootstrap_servers = ['localhost:9092'],
    bootstrap_servers = [SERVER],
    value_serializer = serializer
)

In [ ]:
def get_logs():
    logs = []
    timestamp = str(dt.datetime.now())

    for machine in MACHINES:
        machine_log = {
            'id_machine': machine,
            'temperature': random.randint(50, 150),
            'rpm': random.randint(1500, 2500),
            'timestamp': timestamp
        }
        logs.append(machine_log)

    return logs

In [ ]:
if __name__ == '__main__':
    @repeat(every(INTERVAL).seconds)
    def job():
        logs = get_logs()

        for log in logs:
            producer.send('sensors', log)
            # producer.send(TOPIC, log)
            print(f'Message sent: {log}')

    while True:
        run_pending()
        time.sleep(INTERVAL)